In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from backtest import *
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pprint import PrettyPrinter

In [ ]:
plt.rcParams['figure.figsize'] = [21, 13]
pd.set_option('precision', 10)
pp = PrettyPrinter()

In [ ]:
exchange = 'bybit'
user = 'your_user_name'
settings = load_settings(exchange, user)
s = 'BTCUSD'
n_days = 21

In [ ]:
'''
path = 'historical_data/bybit/agg_trades_futures/BTCUSD/'
dirnames = [f for f in os.listdir(path) if f.endswith('.csv')]
for d in dirnames:
    dft = pd.read_csv(path + d)
    if 'amount' in dft.columns:
        dft.columns = ['qty' if c == 'amount' else c for c in dft.columns]
        print('replacing', path + d)
        dft.to_csv(path + d)
    else:
        print('skipping', path + d)

'''

In [ ]:
#load cache if any
agg_trades = pd.read_csv('btcusdt_agg_trades_bybit_14_days_2021-01-04.csv').set_index('trade_id')

In [ ]:
#otherwise
#agg_trades = await load_trades(exchange, user, s, n_days)

In [ ]:
#age_limit = (time() - 60 * 60 * 24 * 5) * 1000
#adf = agg_trades[agg_trades.timestamp > age_limit]
adf = agg_trades

In [ ]:
# remove consecutive price duplicates
adf = adf[adf.price != adf.price.shift(1)]
n_days = (adf.timestamp.iloc[-1] - adf.timestamp.iloc[0]) / 1000 / 60 / 60 / 24
len(adf), 'n_days', n_days

In [ ]:
# look at jackrabbit results
rdf = pd.concat([pd.read_csv('jackrabbit_results_grid/2021-01-04T19:35:37.csv'),
                 pd.read_csv('jackrabbit_results_grid/2021-01-04T19:11:18.csv')])
rdf.columns = ['ddown_factor', 'ema_spans', 'ema_spread', 'entry_qty_equity_multiplier',
               'grid_spacing', 'initial_equity', 'leverage', 'markup'] + list(rdf.columns[8:])
rdfs = rdf.sort_values('pnl_sum', ascending=[False])
#rdfs = rdfs[rdfs.n_liqs == 0.0]
rdfs.head(30)


In [ ]:
settings = {
    'ema_spans': [5000, 2500],
    'ema_spread': -0.0004,
    'entry_qty_equity_multiplier': 0.002,
    'ddown_factor': 0.01,
    'grid_spacing': 0.003,
    'initial_equity': 0.001,
    'leverage': 100.0,
    'maker_fee': -0.00025,
    'markup': 0.0019,
    'min_qty': 1.0,
    'price_step': 0.5,
    'qty_step': 1.0,
    'symbol': 'BTCUSD',
    'compounding': False,
    'liq_modifier': 0.001,
    'n_days': n_days
}

In [ ]:
# set best jackrabbit result as settings
bs = dict(rdfs.iloc[0])
for k in bs:
    if k in settings:
        if type(bs[k]) == str:
            settings[k] = eval(bs[k].strip())
        else:
            settings[k] = round(bs[k], 10)
        print(k, settings[k])
settings

In [ ]:
# dump settings
# json.dump(settings, open(f'settings/bybit/{user}.json', 'w'), indent=4, sort_keys=True)

In [ ]:
# plotting

def plot_tdf_(df_, tdf_, side_: int = 0):
    df_.loc[tdf_.index[0]:tdf_.index[-1]].price.plot(style='y-')
    if side_ >= 0:
        longs = tdf_[tdf_.side == 'long']
        le = longs[longs.type == 'entry']
        lc = longs[longs.type == 'close']
        ll = longs[longs.type == 'liq']
        le.price.plot(style='b.')
        longs.pos_price.plot(style='b--')
        longs.close_price.plot(style='r--')
        longs.liq_price.plot(style='k--')
        lc.price.plot(style='ro')
        ll.price.plot(style='gx')
    if side_ <= 0:
        shrts = tdf_[tdf_.side == 'shrt']
        se = shrts[shrts.type == 'entry']
        sc = shrts[shrts.type == 'close']
        sl = shrts[shrts.type == 'liq']
        se.price.plot(style='r.')
        shrts.pos_price.plot(style='r--')
        shrts.close_price.plot(style='b--')
        shrts.liq_price.plot(style='k--')
        sc.price.plot(style='bo')
        sl.price.plot(style='gx')

In [ ]:
# prep df for backtesting
df = prep_df(adf, settings)
print(len(df), len(df) / len(adf), )
df[['price', 'bid_thr', 'ask_thr']].iloc[::100].plot()

In [ ]:

settings['ema_spans'] = (50000, 250000)
settings['ema_spread'] = 0.0001
settings['entry_qty_equity_multiplier'] = 0.0
settings['ddown_factor'] = 0.05
settings['grid_spacing'] = 0.005
settings['initial_equity'] = 1.0
settings['leverage'] = 100
#settings['maker_fee'] = -0.00025
settings['markup'] = 0.0019
#settings['min_qty'] = 1.0
#settings['price_step'] = 0.5
#settings['qty_step'] = 1.0
#settings['symbol'] = 'BTCUSD'
settings['compounding'] = False
settings['liq_modifier'] =  0.001
settings['n_days'] = (adf.timestamp.iloc[-1] - adf.timestamp.iloc[0]) / 1000 / 60 / 60 / 24
price_ = df.iloc[0].price
print('price', price_)
print('approx price_step', price_ * settings['grid_spacing'])
print('approx initial qty given initial eq',
      settings['initial_equity'] * settings['entry_qty_equity_multiplier'] * price_)
settings

In [ ]:
# backtest
start_time = time()
trades = backtest(df, settings)
print('time elapsed', time() - start_time)

In [ ]:
# analyze results
tdf = pd.DataFrame(trades).set_index('trade_id')
longs = tdf[tdf.side == 'long']
shrts = tdf[tdf.side == 'shrt']
le = longs[longs.type == 'entry']
lc = longs[longs.type == 'close']
ll = longs[longs.type == 'liq']
se = shrts[shrts.type == 'entry']
sc = shrts[shrts.type == 'close']
sl = shrts[shrts.type == 'liq']

margin_max = tdf.margin_cost.max()
pnl_sum = tdf.pnl.sum()
gain = (margin_max + pnl_sum) / margin_max
n_days = (adf.timestamp.iloc[-1] - adf.timestamp.iloc[0]) / 1000 / 60 / 60 / 24
average_daily_gain = gain ** (1 / n_days)
liqs = tdf[tdf.type == 'liq']
closes = tdf[tdf.type == 'close']
print('margin_max', margin_max)
print('pnl_sum', pnl_sum)
print('gain', gain)
print('n_days', n_days)
print('average_daily_gain', average_daily_gain)
print('n trades', len(tdf))
print('n closes', len(closes))
print('n liqs', len(liqs))

In [ ]:
# visualize behavior
step = 200
i = -step

In [ ]:
i += step
plot_tdf_(df, tdf.iloc[i:i+step])

In [ ]:
tdf.iloc[i:i+step]

In [ ]:
(tdf.qty / tdf.equity).plot()

In [ ]:
tdf.equity_available.plot()

In [ ]:
# cumulative pnl
tdf.pnl.cumsum().plot()

In [ ]:
# liquidations
liqs

In [ ]:
# inspect liquidations
liq_i = 0
liq_iloc = tdf.index.get_loc(liqs.index[liq_i])
iminus = 20
iplus = 20
tdfc = tdf.iloc[max(0, liq_iloc-iminus):min(liq_iloc+iplus, len(tdf) - 1)]
plot_tdf_(df, tdfc)

In [ ]:
tdfc

In [ ]:
qty_abs = tdf.qty.abs().sort_values(ascending=False)
qty_abs.head(10)

In [ ]:
# inspect biggest trades
i = 0
iloc_ = tdf.index.get_loc(qty_abs.index[i])
iminus = 255
iplus = 30
tdfc = tdf.iloc[max(0, iloc_-iminus):min(iloc_+iplus, len(tdf) - 1)]
plot_tdf_(df, tdfc)

In [ ]:
tdfc.head(40)

In [ ]:
31228.0 / 28835.0

In [ ]:
tdfc.loc[1260333:1274857].head(42)

In [ ]:
0.001 * 2**10